In [ ]:
import numpy as np
import os
%matplotlib inline
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
import glob
import re
from tkinter import filedialog as fd
from tkinter import *

In [ ]:
def getFilePathFromDialog(_initialdir):
    root = Tk()
    FilePath = fd.askopenfilename(initialdir = _initialdir)
    root.destroy()
    return FilePath

def getDirectoryPathFromDialog(_initialdir):
    root = Tk()
    Directory = fd.askdirectory(initialdir = _initialdir)
    root.destroy()
    return Directory

In [ ]:
def readFile(path, skiprows):
    return np.loadtxt(path, dtype=float, skiprows=skiprows)

def readFileFromDialog(skiprows):
    return readFile(getFilePathFromDialog(dataPath),skiprows)

In [ ]:
def readParticleConfig(path,ConfigurationNumber):
    TotalNumberOfParticles = int(re.search(r'(?<=N=)[\d]*',path).group())
    with open(path,'r') as F:
        Data = F.read()
    RowOffset = 1+ConfigurationNumber*(TotalNumberOfParticles+2)
    print("RowOffset: ",RowOffset)
    Data = Data.strip()
    FirstRow = Data.split(sep='\n')[RowOffset]
    NumberOfAParticles = int(FirstRow.split()[5].strip('|'))
    NumberOfBParticles = int(FirstRow.split()[7].strip('|'))
    ListOfRows = Data.split(sep='\n')[RowOffset+1:RowOffset+1+TotalNumberOfParticles]
    if (len(ListOfRows) > 0):
        NumberOfColumns = len(ListOfRows[0].split()) - 1
    else:
        NumberOfColumns = 0
    APositions = np.empty((NumberOfAParticles, NumberOfColumns))
    BPositions = np.empty((NumberOfBParticles, NumberOfColumns))
    AIndex = 0
    BIndex = 0
    
    for i in range(len(ListOfRows)):
        if (ListOfRows[i].split()[NumberOfColumns] == 'A'):
            for j in range(NumberOfColumns):
                APositions[AIndex,j] = ListOfRows[i].split()[j]
            AIndex += 1
        else:
            for j in range(NumberOfColumns):
                BPositions[BIndex,j] = ListOfRows[i].split()[j]
            BIndex += 1
    return APositions, BPositions

def readParticleConfigFromDialog(ConfigurationNumber):
    return readParticleConfig(getFilePathFromDialog(dataPath),ConfigurationNumber)

In [ ]:
def LinearFunction(x,m,n):
    return m*x+n

In [ ]:
dataPath = getDirectoryPathFromDialog('')

## Energy series

In [ ]:
EnergySeries = readFileFromDialog(1)

In [ ]:
Fig, Ax = plt.subplots(2,2,squeeze=False,figsize=(15,9))
Ax[0,0].plot(EnergySeries[:1000,0],label='U')
Ax[0,0].plot(EnergySeries[:1000,1],label='T')
Ax[0,0].plot(EnergySeries[:1000,2],label='H=U+T')
Ax[0,0].legend()
Ax[0,1].plot(EnergySeries[:,2],label='H=U+T')
Ax[0,1].set_ylabel('H=T+U')
MeanH = np.mean(EnergySeries[:,2])
print(MeanH)
print(np.std(EnergySeries[:,2]))
Ax[0,1].plot(MeanH*np.ones(len(EnergySeries[:,2])))
n,bins,patches = Ax[1,1].hist(EnergySeries[:,2],bins = 41, density = True)
Ax[1,1].vlines(MeanH,0,np.max(n),color='darkorange')